In [2]:
import ccxt
import requests
from binance.client import Client
from binance.exceptions import BinanceAPIException
from datetime import datetime, timedelta
import time
import hashlib
import hmac
import pandas
import json


In [30]:
with open("nested_dataframes.json") as config_file:
    nested_bal=json.load(config_file)
len(nested_bal)

44

In [31]:
first_key=list(nested_bal.keys())[0]
last_key=list(nested_bal.keys())[-1]

In [32]:
first_key, last_key

('2024-06-21 21:27:51', '2024-06-21 21:39:59')

In [33]:
first_bal=nested_bal[first_key]


In [34]:
first_bal

[{'Asset': 'ETH', 'Free': 2.94, 'Locked': 0.0, 'Total': 100.94},
 {'Asset': 'BTC', 'Free': 1.02604, 'Locked': 0.0, 'Total': 1.02604},
 {'Asset': 'LTC', 'Free': 5.0, 'Locked': 0.0, 'Total': 5.0},
 {'Asset': 'BNB', 'Free': 1.0, 'Locked': 0.0, 'Total': 1.0},
 {'Asset': 'USDT', 'Free': 1157.0442307, 'Locked': 0.0, 'Total': 1157.0442307},
 {'Asset': 'TRX', 'Free': 4386.0, 'Locked': 0.0, 'Total': 4386.0},
 {'Asset': 'XRP', 'Free': 948.0, 'Locked': 0.0, 'Total': 948.0},
 {'Asset': 'NEO', 'Free': 33.0, 'Locked': 0.0, 'Total': 33.0},
 {'Asset': 'QTUM', 'Free': 141.0, 'Locked': 0.0, 'Total': 141.0},
 {'Asset': 'EOS', 'Free': 617.0, 'Locked': 0.0, 'Total': 617.0},
 {'Asset': 'SNT', 'Free': 14258.0, 'Locked': 0.0, 'Total': 14258.0},
 {'Asset': 'BNT', 'Free': 606.0, 'Locked': 0.0, 'Total': 606.0},
 {'Asset': 'GAS', 'Free': 98.0, 'Locked': 0.0, 'Total': 98.0},
 {'Asset': 'LRC', 'Free': 1850.0, 'Locked': 0.0, 'Total': 1850.0},
 {'Asset': 'ZRX', 'Free': 931.0, 'Locked': 0.0, 'Total': 931.0},
 {'Asset'

In [35]:
last_bal=nested_bal[last_key]

In [36]:
def list_to_dict(asset_list):
    """Convert a list of asset dictionaries to a dictionary with asset names as keys."""
    return {item['Asset']: item for item in asset_list}

def subtract_balance_sheets(sheet1, sheet2):
    """Subtract the balances of two balance sheets."""
    sheet1_dict = list_to_dict(sheet1)
    sheet2_dict = list_to_dict(sheet2)
    
    result_dict = {}
    
    all_assets = set(sheet1_dict.keys()).union(set(sheet2_dict.keys()))
    
    for asset in all_assets:

        free1 = sheet1_dict.get(asset, {'Free': 0.0})['Free']
        locked1 = sheet1_dict.get(asset, {'Locked': 0.0})['Locked']
        total1 = sheet1_dict.get(asset, {'Total': 0.0})['Total']
        
        free2 = sheet2_dict.get(asset, {'Free': 0.0})['Free']
        locked2 = sheet2_dict.get(asset, {'Locked': 0.0})['Locked']
        total2 = sheet2_dict.get(asset, {'Total': 0.0})['Total']
        if ((total1 -total2)!=0):
            result_dict[asset] = {
                'Asset': asset,
                'Free': free1 - free2,
                'Locked': locked1 - locked2,
                'Total': total1 - total2
            }
    
    return list(result_dict.values())

In [43]:
subtract_balance_sheets(first_bal,last_bal)

[{'Asset': 'ETH', 'Free': 0.0, 'Locked': 0.0, 'Total': 98.0}]

In [41]:
def return_balance_sheet_change(filepath,destinationpath):
    with open(filepath) as config_file:
        nested_bal=json.load(config_file)
    first_key=list(nested_bal.keys())[0]
    last_key=list(nested_bal.keys())[-1]
    first_bal=nested_bal[first_key]
    last_bal=nested_bal[last_key]
    net_bal=subtract_balance_sheets(first_bal,last_bal)
    with open(destinationpath, 'w') as file:
        json.dump(net_bal, file)
    return net_bal


In [44]:
return_balance_sheet_change("nested_dataframes.json","net_bal_change.json")

[]